# Importation des données

In [12]:
import os
import string
import re
import math
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)

import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle

os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')

with open("data/corpus_trie.file", "rb") as f:
    corpus = pickle.load(f) 
phrases = [phrase.split() for phrase in corpus]

In [13]:
words = [item for sublist in phrases for item in sublist]
print(type(words))
vocabulary = set(words)
print("Nombre de mots :", len(words))
print("Taille du vocabulaire :", len(vocabulary))

<class 'list'>
Nombre de mots : 31394494
Taille du vocabulaire : 634691


In [ ]:
# Pour changer la taille des graphiques :
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["font.size"] = 20

fdist = nltk.FreqDist(words)
print("Les 10 mots les plus communs sont :")
print(fdist.most_common(10))
fdist.plot(50)

In [ ]:
# Fonction pour mettre à jour le graphique en direct
def live_plot(data, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.plot(data)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.show();

# Sampling rate et negative sampling
On va simplifier un peu le corpus en enlevant certains mots. Pour cela on va faire un sous-échantillonnage du corpus pour supprimer certains mots. 

Pour chaque mot $w_i$ on note $z(w_i)$ la proportion d'apparition de ce mot, c'est-à-dire le rapport entre le nombre de fois que ce mot apparait et le nombre total de mots. La probabilité de garder un mot le mot $w_i$ est :
$$
\mathbb P(w_i) = \left(\sqrt{\frac{z(w_i)}{q}} + 1 \right)
\times
\frac{q}{z(w_i)}
$$
Le paramètre $q$ est appelé "sample" – échantillonnage – contrôle le nombre de sous-échantillonnages. La valeur par défaut est 0,001.

In [6]:
def calcul_proba(x):
    result = (sqrt(x)+1)*(1/x)
    return(result)
calcul_proba_v = np.vectorize(calcul_proba) # Pour vectoriser la fonction
# Fonction pour créer l'échantillon
def creer_echantillon(phrases, vocabulary , probabilities_subsampling,  window = 2):
    #Sub-sampling
    nouveau_corpus = [] 
    for phrase in phrases: #on parcourt tous les articles du corpus
        nouveau_corpus.append([]) #on crée une sous liste à chaque nouvel article
        for word in phrase: #et pour tous les mots de l'article
        # Les mots à supprimer sont les mots tels que la loi générée U([0,1]) soit > proba
        # On garde donc les mots si U([0,1]) <= proba
            proba_w = probabilities_subsampling[vocabulary.index(word)]
            if np.random.uniform(low=0.0, high=1.0) <= proba_w: # Je garde le mot
                nouveau_corpus[-1].append(word) 
    phrases = [phrase for phrase in nouveau_corpus if len(phrase)>1] # On enlève les phrases avec 1 seul mot
    test_sample = []
    for phrase in phrases:
        # Pour chaque phrase on prend au hasard un mot focus et un mot contexte
        focus = list(range(0, len(phrase)))
        focus = random.choice(focus)
        i = focus
        index_i = vocabulary.index(phrase[i])
        i_contexte = list(range(max(i-window,0), min(i+window+1, len(phrase))))
        i_contexte.remove(i)
        i_contexte = random.choice(i_contexte)
        j = i_contexte
        index_j = vocabulary.index(phrase[j])
        test_sample.append([index_i, index_j])
    return(test_sample)

sample = 0.001
fdist = nltk.FreqDist(words)
vocabulary = list(set(words))
proportion = np.array([(fdist[w]/ (len(words) * sample)) for w in vocabulary])
p_subsampling = calcul_proba_v(proportion) # C'est le vecteur contenant les proba de sub-sampling

# Algorithme avec softmax
Si on note $\theta$ le paramètre à estimer, $L(\theta)$ la fonction de perte et $\eta$ le taux d'apprentissage (*learning rate*) alors :
$$
\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta^{(t)})
$$
C'est ce que l'on appelle la descente de gradient. On va appliquer cette méthode à chaque étape pour la matrice d'input et la matrice d'output.

In [13]:
dim = 10
epoch = 10
learning_rate = 0.01

# Attention: torch.rand génère une loi uniforme et torch.randn une loi normale
input = torch.randn(len(vocabulary), dim)
output = torch.randn(len(vocabulary), dim)
input = autograd.Variable(input, requires_grad=True)
output = autograd.variable(output, requires_grad=True)

loss_tot = []
temps_par_epoch = []

start = time.time()
for i in range(epoch):
    #print(i)
    loss_val = 0
    start_epoch = time.time()
    test_sample = creer_echantillon(phrases, vocabulary, p_subsampling)
    for focus, context in test_sample:
        # Multiplication matricielle: 
        data = torch.matmul(input[focus,], torch.t(output))
        #log_probs = F.log_softmax(data, dim=0)
        #loss = F.nll_loss(log_probs.view(1,-1), torch.tensor([context]))
        # Il semble que cela combine les deux précédentes fonctions : 
        # https://pytorch.org/docs/stable/nn.functional.html#cross-entropy
        loss = F.cross_entropy(data.view(1,-1), torch.tensor([context]))
        #print(loss)
        loss_val += loss.data
        # Pour ensuite dériver les matrices par rapport à la loss
        loss.backward()
        
        # Il faut modifier juste le .data pour ne pas perdre la structure
        input.data = input.data - learning_rate * input.grad.data
        output.data = output.data - learning_rate * output.grad.data
        
        input.grad.data.zero_()
        output.grad.data.zero_()
    
    end_epoch = time.time()
    temps_par_epoch.append(end_epoch - start_epoch)
    loss_val = loss_val / len(vocabulary)
    loss_tot.append(loss_val)
    live_plot(loss_tot)
end = time.time()
print(round((end - start)/60, 2))
#print(input)        
#plt.plot(loss_tot)

KeyboardInterrupt: 

In [ ]:
matrice_finale = (input + output)/2
with open("data/matrice_finale_softmax.file", "wb") as f:
    pickle.dump(matrice_finale, f, pickle.HIGHEST_PROTOCOL)